In [ ]:
import databutton as db
import requests
from bs4 import BeautifulSoup
import openai
import cohere
import streamlit as st

In [ ]:
def scrape_text(url):
    # Send a GET request to the URL
    response = requests.get(url)

    # If the GET request is successful, the status code will be 200
    if response.status_code == 200:
        # Get the content of the response
        page_content = response.content

        # Create a BeautifulSoup object and specify the parser
        soup = BeautifulSoup(page_content, "html.parser")

        # Get the text of the soup object
        text = soup.get_text()

        # Return the text
        return text
    else:
        return "Failed to scrape the website"

In [ ]:
def mermaid_chart(mindmap_code):
    html_code = f"""
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.1/css/all.min.css">
    <div class="mermaid">{mindmap_code}</div>
    <script src="https://cdn.jsdelivr.net/npm/mermaid/dist/mermaid.min.js"></script>
    <script>mermaid.initialize({{startOnLoad:true}});</script>
    """
    return html_code

In [ ]:
def run_models(input_text):
    # Note: content is hidden due to length of text, you can add user and assistant roles
    response = openai.ChatCompletion.create(
        engine="GPT35turbo",
        messages=[
            {
                "role": "system",
                "content": "You are generating a mindmap for someone who has ADHD...",
            },
            {"role": "user", "content": input_text},
        ],
        temperature=0.2,
        max_tokens=400,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
    )

    return response

In [ ]:
def summarise(input_text):
    co = cohere.Client(
        db.secrets.get("lox3xFZpdEh4xDt2IFcW6xEo4zh9RtqADPeZrw0r")
    )
    response = co.summarize(
        text=input_text,
        length="long",
        format="paragraph",
        model="summarize-xlarge",
        additional_command="What are th"

In [ ]:
import databutton as db
import streamlit as st
from streamlit.components.v1 import html
from utils import scrape_text, mermaid_chart, run_models, summarise

# UI
st.title("AI Generated Mermaid.js Mindmap")

# Creating a form for both caching and clean input box on run
form = st.form("Form to run", clear_on_submit=True)

url = form.text_input("Enter your URL below:", placeholder="Paste any URL of your choice")

if form.form_submit_button("Generate Mermaid Mindmap Visual"):
    text = scrape_text(url)
    input_text = "Generate a Mermaid.js mindmap only using the text below:\n" + text
    with st.expander("See full article"):
        st.write(text)
    with st.spinner("Generating Cohere AI Summary "):
        summary = summarise(text)
        st.write("### Cohere AI Generated Summary")
        st.write(summary)

    with st.spinner("Generating Mermaid Code"):
        out = run_models(input_text)

    mindmap_code = out["choices"][0]["message"]["content"][10:-3]
    with st.expander("See OpenAI Generated Mermaid Code"):
        st.code(mindmap_code)

    # Use the html function to display the Mermaid.js diagram
    html(mermaid_chart(mindmap_code), width=1500, height=1500)